In [ ]:
import socket
import threading
import os

s = socket.socket()
print("Socket created")

s.bind(('localhost', 9996))
s.listen(5)

print("Waiting for connections...")

clients = []

ALLOWED_EXT = [".txt", ".jpg", ".pdf"]
BANNED_WORDS = ["spam", "sensitive", "hack"]

def handle_client(c, addr):
    print("Connected with", addr)
    clients.append(c)

    while True:
        try:
            data_from_client = c.recv(1024).decode()

            # MESSAGE HANDLING
            if data_from_client.startswith("MSG:"):
                msg = data_from_client[4:]

                
                if not data_from_client or data_from_client.lower() == "exit":
                    print("Client disconnected:", addr)
                    if c in clients:
                        clients.remove(c)
                    c.close()
                    break

                if any(word in msg.lower() for word in BANNED_WORDS):
                    c.send("Message blocked due to inappropriate content".encode())
                    
                else:
                    print(f"{addr} says:", msg)
                    
                    for client in clients:
                        if client != c:
                            client.send(f"MSG from {addr}: {msg}".encode())
                          

            # FILE HANDLING
            elif data_from_client.startswith("FILE:"):
                filename = data_from_client[5:]
                ext = os.path.splitext(filename)[1]

                if ext not in ALLOWED_EXT:
                    c.send("File type not allowed".encode())
                    
                else:
                    print(f"{addr} sending file:", filename)

                    file_data = c.recv(4096)

                    print(f"{addr} sending file:", filename)
                    
                    for client in clients:
                        if client != c:
                            client.send(f"File from {addr}: {filename}".encode())
                            

        except:
            if c in clients:
                clients.remove(c)
            c.close()
            break

while True:
    c, addr = s.accept()
    threading.Thread(target=handle_client, args=(c, addr)).start()


Socket created
Waiting for connections...
Connected with ('127.0.0.1', 55738)
Connected with ('127.0.0.1', 55748)
('127.0.0.1', 55738) sending file: Q3ex.pdf
('127.0.0.1', 55738) sending file: Q3ex.pdf
xyz
('127.0.0.1', 55748) says: hi
Connected with ('127.0.0.1', 55760)
Connected with ('127.0.0.1', 55856)
Connected with ('127.0.0.1', 56182)
Connected with ('127.0.0.1', 56189)
Connected with ('127.0.0.1', 56193)
('127.0.0.1', 56182) says: hi
hahahah
hahahah
('127.0.0.1', 56189) sending file: Q3ex.pdf
('127.0.0.1', 56189) sending file: Q3ex.pdf
xyz
xyz
('127.0.0.1', 56182) says: exit
hahahah
('127.0.0.1', 56193) says: exit
Connected with ('127.0.0.1', 56299)
('127.0.0.1', 56299) says: exit
